In [1]:
import math

In [2]:
class MazeSolver:
    def __init__(self, maze, start, goal):
        self.maze = maze
        self.start = start
        self.goal = goal 
        self.nodes_expanded = 0 
        self.rows = len(maze)
        self.cols = len(maze[0])

    def get_neighbors(self,  node):
        x, y = node 
        neighbors = []
        for dx, dy in [(-1, 0), (1, 0), (0, 1), (0, -1)]:
            nx, ny = x + dx, y + dy 
            if 0 <= nx < self.rows and 0 <= ny < self.cols and self.maze[nx][ny] == 0:
                neighbors.append((nx, ny))
        return neighbors 
    
    def h(self, node):
        return abs(node[0] - self.goal[0]) + abs(node[1] - self.goal[1])
    
    def rbfs_search(self):
        self.nodes_expanded = 0
        result, f_final = self.rbfs(self.start, 0, math.inf, [])
        return result
    
    def rbfs(self, node, g, f_limit, path):
        self.nodes_expanded += 1 
        current_path = path + [node]

        if node == self.goal:
            return current_path, g
        
        neighbors = self.get_neighbors(node)
        if not neighbors:
            return None, math.inf
        
        succ = []
        for neighbor in neighbors:
            if neighbor in path: continue 
            g_neighbor = g + 1
            f_neighbor = max(g_neighbor + self.h(neighbor), g + self.h(node))
            succ.append({'node': neighbor, 'f': f_neighbor, 'g': g_neighbor})
        
        if not succ:
            return None, math.inf 
        
        while True: 
            succ.sort(key=lambda x: x['f'])
            best = succ[0]

            if best['f'] > f_limit:
                return None, best['f']
            
            alt = succ[1]['f'] if len(succ) > 1 else f_limit 

            result, best['f'] = self.rbfs(best['node'], best['g'], min(f_limit, alt), current_path)

            if result is not None:
                return result, best['f']

In [3]:
grid1 = [
    [0, 0, 0, 0, 0],
    [1, 1, 0, 1, 0],
    [0, 0, 0, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0]
]

start_pos = (0, 0)
goal_pos = (4, 4)

solver = MazeSolver(grid1, start_pos, goal_pos)
path_found = solver.rbfs_search()

print("--- RBFS Results ---")
if path_found:
    print(f"Path Found: {path_found}")
else:
    print("No path found.")
print(f"Nodes Expanded: {solver.nodes_expanded}")

--- RBFS Results ---
Path Found: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (2, 4), (3, 4), (4, 4)]
Nodes Expanded: 11


In [5]:
class IDAStarSolver:
    def __init__(self, maze, start, goal):
        self.maze = maze 
        self.start = start
        self.goal = goal 
        self.nodes_expanded = 0
        self.rows = len(maze)
        self.cols = len(maze[0])

    def h(self, node):
        return abs(node[0] -self.goal[0]) + abs(node[1] - self.goal[1])
    
    def get_neighbors(self, node):
        x, y = node 
        neighbors = []
        for dx, dy in [(-1, 0), (1, 0), (0, 1), (0, -1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.rows and 0 <= ny < self.cols and self.maze[x][y] == 0:
                neighbors.append((nx, ny))
        return neighbors 
    
    def solve(self):
        threshold = self.h(self.start)
        path = [self.start]

        while True:
            result, t = self.search(path, 0, threshold)
            if result == "found":
                return path, threshold 
            if t == math.inf:
                return None, None 
            threshold = t
    
    def search(self, path, g, threshold):
        node = path[-1]
        f = g + self.h(node)

        if f > threshold:
            return "cutoff", f
        
        if node == self.goal:
            return "found", f 
        
        min_threshold = math.inf 
        self.nodes_expanded += 1 

        for neighbor in self.get_neighbors(node):
            path.append(neighbor)
            result, t = self.search(path, g + 1, threshold)
            if result == "found":
                return "found", t
            if t < min_threshold:
                min_threshold = t
            path.pop()
        return "cutoff", min_threshold

In [6]:
maze2 = [
    [0, 0, 0, 0, 0],
    [1, 1, 0, 1, 1],
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0]
]

start_node = (0, 0)
goal_node = (4, 4)

solver = IDAStarSolver(maze2, start_node, goal_node)
final_path, final_cost = solver.solve()

# Output Results
print("--- IDA* Search Results ---")
if final_path:
    print(f"Path found: {final_path}")
    print(f"Total Path Cost: {final_cost}")
else:
    print("No path found.")
print(f"Total nodes expanded: {solver.nodes_expanded}")

--- IDA* Search Results ---
Path found: [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (2, 3), (2, 4), (3, 4), (4, 4)]
Total Path Cost: 8
Total nodes expanded: 12


In [7]:
class SMAStarSolver:
    def __init__(self, maze, start, goal, max_mem):
        self.maze = maze 
        self.start = start 
        self.goal = goal 
        self.max_mem = max_mem
        self.nodes_expanded = 0 
        self.rows = len(maze)
        self.cols = len(maze[0])
        self.memory = {}

    def h(self, node):
        return abs(node[0] - self.goal[0]) + abs(node[1] - self.goal[1])
    
    def get_neighbors(self, node):
        x, y = node 
        neighbors = []

        for dx, dy in [(-1, 0), (1, 0), (0, 1), (0, -1)]:
            nx, ny = x + dx, y + dy 
            if 0 <= nx < self.rows and 0 <= ny < self.cols and self.maze[nx][ny] == 0:
                neighbors.append((nx, ny))
        return neighbors 
    
    def solve(self):
        root_f = self.h(self.start)
        root = {'pos': self.start, 'g': 0, 'f': root_f, 'parent': None}
        self.memory = {self.start: root}

        while True:
            if not self.memory:
                return None, 0 
            
            best_pos = min(self.memory, key=lambda p: (self.memory[p]['f'], -self.memory[p]['g']))
            best_node = self.memory[best_pos]

            if best_node['pos'] == self.goal:
                return self.reconstruct_path(best_node), best_node['f']
            
            self.nodes_expanded += 1
            neighbors = self.get_neighbors(best_node['pos'])

            for move in neighbors:
                if move not in self.memory:
                    if len(self.memory) > self.max_mem:
                        worst_pos = max(self.memory, key=lambda p: (self.memory[p]['f'], self.memory[p]['g']))
                        del self.memory[worst_pos]
                    g_new = best_node['g'] + 1
                    f_new = max(best_node['f'], g_new + self.h(move))
                    self.memory[move] = {'pos': move, 'g': g_new, 'f': f_new, 'parent': best_node}
            
    def reconstruct_path(self, node):
        path = []
        while node:
            path.append(node['pos'])
            node = node['parent']
        return path[::-1]

In [8]:
maze3 = [
    [0, 0, 0, 0, 0],
    [1, 1, 0, 1, 0],
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0]
]

solver = SMAStarSolver(maze3, (0, 0), (4, 4), max_mem=10)
path, cost = solver.solve()

print("--- SMA* Search Results ---")
print(f"Path: {path}")
print(f"Nodes Expanded: {solver.nodes_expanded}")

--- SMA* Search Results ---
Path: [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (2, 3), (2, 4), (3, 4), (4, 4)]
Nodes Expanded: 8
